In [1]:
import requests
import pprint
from datetime import datetime, timezone, timedelta

In [2]:
today = datetime.now().strftime('%Y-%m-%d')

url = f"https://statsapi.mlb.com/api/v1/schedule?sportId=1&date={today}&hydrate=probablePitcher(note,stats,person),decisions&language=en"
response = requests.get(url)
data = response.json()

In [3]:
# Perhaps change this to https://statsapi.mlb.com/api/v1/schedule?sportId=1

games = data['dates'][0]['games']
game_ids = [game['gamePk'] for game in games]

In [4]:
time_left = (datetime.strptime(games[0]['gameDate'],"%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=timezone.utc) - datetime.now(timezone.utc))
year = datetime.now().year

In [5]:
streaks = {}

# Can possibly put both league IDs into one URL
# Potentiall can hydrate
for league in [103,104]:
    url = f"https://statsapi.mlb.com/api/v1/standings?leagueId={league}&season={year}&type=regularSeason"
    response = requests.get(url)
    data = response.json()   
    for division in data['records']:
        for team in division['teamRecords']:
            streaks[team['team']['id']] = team['streak']['streakCode']

In [6]:
time_left = (datetime.strptime(games[0]['gameDate'],"%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=timezone.utc) - datetime.now(timezone.utc))
year = datetime.now().year

if time_left.days < 0:
    print("Baseball has started!")
else:
     hours = time_left.seconds // 3600
     minutes = (time_left.seconds % 3600) // 60
     seconds = (time_left.seconds % 60)
     print(f"Baseball starts in {hours} hours, {minutes} minutes and {seconds} seconds")

Baseball has started!


In [7]:
def pitcher_stats(pitcher_id):
    url = f"https://statsapi.mlb.com/api/v1/people/{pitcher_id}/stats?stats=season&group=pitching"
    response = requests.get(url)
    data = response.json()
    stats = data['stats'][0]['splits'][0]['stat']
    stats['name'] = data['stats'][0]['splits'][0]['player']['fullName']
    try:
        stats['team'] = data['stats'][0]['splits'][0]['team']['name']
    except:
        stats['team'] = data['stats'][0]['splits'][1]['team']['name']
    return(stats)

In [8]:
def game_details(game_id):
    url = f"https://statsapi.mlb.com/api/v1.1/game/{game_id}/feed/live"
    response = requests.get(url)
    data = response.json()
    return(data)

In [9]:
game_ids = [game['gamePk'] for game in games]
for game in games:
    game_id = game['gamePk']
    teams = game['teams']
    away_team = teams['away']['team']['name']
    home_team = teams['home']['team']['name']
    away_id = teams['away']['team']['id']
    home_id = teams['home']['team']['id']
    try:
        home_pitcher_name = teams['home']['probablePitcher']['fullName']
    except:
        home_pitcher_name = "TBD"
    try:
        home_pitcher_stats = pitcher_stats(teams['home']['probablePitcher']['id'])
    except:
        home_pitcher_stats = {}
    try:
        away_pitcher_name = teams['away']['probablePitcher']['fullName']
    except:
        away_pitcher_name = "TBD"
    try:
        away_pitcher_stats = pitcher_stats(teams['away']['probablePitcher']['id'])
    except:
        away_pitcher_stats = {}
    home_record = game['teams']['home']['leagueRecord']
    away_record = game['teams']['away']['leagueRecord']
    game_time = datetime.strptime(game['gameDate'],'%Y-%m-%dT%H:%M:%SZ').replace(tzinfo=timezone.utc).astimezone(tz=None)
    if game['status']['abstractGameState'] == 'Preview' or game['status']['detailedState'] == 'Warmup':
        print(f"{game_time.strftime('%I:%M %p')}:")
        try:
            print(f"{away_team} ({away_record['wins']}-{away_record['losses']}, {streaks[away_id]}) -- {away_pitcher_name} ({away_pitcher_stats['wins']}-{away_pitcher_stats['losses']}, {away_pitcher_stats['era']} ERA)")
        except:
            print(f"{away_team} ({away_record['wins']}-{away_record['losses']}, {streaks[away_id]}) -- {away_pitcher_name}")
        print("vs.")
        try:
            print(f"{home_team} ({home_record['wins']}-{home_record['losses']}, {streaks[home_id]}) -- {home_pitcher_name} ({home_pitcher_stats['wins']}-{home_pitcher_stats['losses']}, {home_pitcher_stats['era']} ERA)")
        except:
            print(f"{home_team} ({home_record['wins']}-{home_record['losses']}, {streaks[home_id]}) -- {home_pitcher_name}")
    
    elif game['status']['abstractGameState'] == 'Live':
        url = f"https://statsapi.mlb.com/{game['link']}"
        response = requests.get(url)
        data = response.json()

        home_score = game['teams']['home']['score']
        away_score = game['teams']['away']['score']
        inning = f"{data['liveData']['linescore']['inningHalf']} {data['liveData']['linescore']['currentInningOrdinal']}"
        print(f"{inning}:")
        print(f"{away_team} {away_score}, {home_team} {home_score}")
        
        print(f"{data['liveData']['plays']['currentPlay']['matchup']['pitcher']['fullName']} pitching to {data['liveData']['plays']['currentPlay']['matchup']['batter']['fullName']}.")
        try:
            print(data['liveData']['plays']['currentPlay']['result']['description'])
            try:
                print(f"Exit Velocity: {data['liveData']['plays']['currentPlay']['playEvents'][-1]['hitData']['launchSpeed']} MPH")
            except: pass
        except:
            try:
                print(data['liveData']['plays']['allPlays'][-2]['result']['description']) 
            except:
                print("Start of game")
            try:
                print(f"Exit Velocity: {data['liveData']['plays']['allPlays'][-2]['playEvents'][-1]['hitData']['launchSpeed']} MPH")
            except: pass
                
        url = f"https://statsapi.mlb.com/api/v1/game/{game_id}/winProbability"
        response = requests.get(url)
        data = response.json()
        probability = round(data[-1]['homeTeamWinProbability'],1)
        if probability < 50:
            print(f"The {away_team} have a {100-probability}% chance of winning.")
        else:
            print(f"The {home_team} have a {probability}% chance of winning.")

    elif game['status']['abstractGameState'] == 'Final':
        home_score = game['teams']['home']['score']
        away_score = game['teams']['away']['score']
        print("Final")
        print(f"{away_team}: {away_score} | {away_record['wins']}-{away_record['losses']}, {streaks[away_id]}")
        print(f"{home_team}: {home_score} | {home_record['wins']}-{home_record['losses']}, {streaks[home_id]}")

        url = f"https://statsapi.mlb.com/api/v1/game/{game_id}/content"
        response = requests.get(url)
        data = response.json()
        try:
            print(data['editorial']['recap']['mlb']['headline'])
        except: pass
        try:
            print("Play of the game:", data['highlights']['highlights']['items'][0]['playbacks'][0]['url'])
        except: pass
            
        try:
            wp = game['decisions']['winner']['fullName']
            lp = game['decisions']['loser']['fullName']
            try:
                sv = game['decisions']['save']['fullName']
                print(f"W: {wp} || L: {lp} || SV: {sv}")
            except:
                print(f"W: {wp} || L: {lp}")
        except: pass

        url = f"https://statsapi.mlb.com/api/v1/game/{game_id}/boxscore"
        response = requests.get(url)
        data = response.json()
        for player in data['topPerformers']:
            name = player['player']['person']['fullName']
            try:
                stats = player['player']['stats']['pitching']['summary']
            except: 
                stats = player['player']['stats']['batting']['summary']    
            print(f"{name}: {stats}")            
        
    print("--------------------------")

Top 4th:
Texas Rangers 3, Baltimore Orioles 0
Charlie Morton pitching to Kyle Higashioka.
Alejandro Osuna out on a sacrifice fly to center fielder Cedric Mullins. Evan Carter scores.
Exit Velocity: 102.8 MPH
The Texas Rangers have a 81.0% chance of winning.
--------------------------
06:40 PM:
Athletics (32-48, L2) -- Luis Severino (2-7, 4.42 ERA)
vs.
Detroit Tigers (49-30, W1) -- Tarik Skubal (8-2, 2.06 ERA)
--------------------------
Bottom 3rd:
Toronto Blue Jays 2, Cleveland Guardians 0
Eric Lauer pitching to José Ramírez.
José Ramírez flies out to center fielder Myles Straw.
Exit Velocity: 96.8 MPH
The Toronto Blue Jays have a 71.2% chance of winning.
--------------------------
Top 2nd:
Atlanta Braves 0, New York Mets 0
Frankie Montas pitching to Ozzie Albies.
Drake Baldwin grounds out, second baseman Ronny Mauricio to first baseman Pete Alonso.
Exit Velocity: 94.7 MPH
The New York Mets have a 52.3% chance of winning.
--------------------------
Top 2nd:
New York Yankees 0, Cincinna